In [1]:
import cv2
import numpy as np
import os

# 定义样本图片目录和样本数量
pos_img_dir = "D:/Content_Secu/INRIAPerson/INRIAPerson/Train/pos"  # 正样本图片目录
neg_img_dir = "D:/Content_Secu/INRIAPerson/INRIAPerson/Train/neg"  # 负样本图片目录
pos_samples = 2400  # 指定正样本数量
neg_samples = 12000  # 指定负样本数量
central_crop = True  # 是否对正样本进行中心裁剪

model_path = "svm_hog.xml"  # 模型保存路径

# 创建HOG描述符
hog = cv2.HOGDescriptor()

def compute_hog_features(img_dir, num_samples, img_size=(64, 128), central_crop=False):
    features = []
    labels = []
    for img_name in sorted(os.listdir(img_dir))[:num_samples]:
        img_path = os.path.join(img_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue  # 如果图像未正确加载，则跳过
        if central_crop:
            img = img[16:-16, 16:-16]  # 假设原图是96x160，裁剪至64x128
        img = cv2.resize(img, img_size)  # 确保图像是一致的尺寸
        descriptor = hog.compute(img)
        if descriptor is not None:
            descriptor = descriptor.reshape(-1)
            features.append(descriptor)
            label = 1 if img_dir == pos_img_dir else -1
            labels.append(label)
    return np.array(features), np.array(labels)

# 计算正样本和负样本的HOG特征
pos_features, pos_labels = compute_hog_features(pos_img_dir, pos_samples, central_crop=central_crop)
neg_features, neg_labels = compute_hog_features(neg_img_dir, neg_samples, central_crop=False)

# 合并特征和标签
features = np.vstack((pos_features, neg_features))
labels = np.hstack((pos_labels, neg_labels))

# 训练SVM分类器
svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setC(0.01)
svm.train(features, cv2.ml.ROW_SAMPLE, labels)
svm.save(model_path)
print("SVM training completed and model saved.")


SVM training completed and model saved.


In [2]:
import cv2
import numpy as np

def get_svm_detector(svm):
    sv = svm.getSupportVectors()
    rho, _, _ = svm.getDecisionFunction(0)
    sv = np.transpose(sv)
    return np.append(sv, [[-rho]], 0)

# 加载训练好的SVM模型
model_path = "svm_hog.xml"
svm = cv2.ml.SVM_load(model_path)

# 创建HOG描述符
win_size = (64, 128)
block_size = (16, 16)
block_stride = (8, 8)
cell_size = (8, 8)
n_bins = 9
hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, n_bins)

# 设置HOG描述符的检测器为训练好的SVM模型
hog.setSVMDetector(get_svm_detector(svm))

# 读取测试图片
test_img_path = "D:/Content_Secu/INRIAPerson/INRIAPerson/Test/pos/person_204.png"
img = cv2.imread(test_img_path)

# 在测试图片上进行行人检测
(rects, _) = hog.detectMultiScale(img, winStride=(8, 8), padding=(16, 16), scale=1.05)

# 画出检测到的行人
for (x, y, w, h) in rects:
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# 显示结果
cv2.imshow("Detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
